In [44]:
import tkinter as tk
from tkinter import simpledialog, Canvas, Frame, Button, Label, Scrollbar, Listbox, Toplevel, Entry, Menu
from PIL import Image, ImageTk, ImageDraw
import os
from datetime import datetime
import pandas as pd

In [45]:
class CustomDialog(Toplevel):
    """自訂對話框，用於輸入或修改註解並選擇類型。"""
    def __init__(self, parent, title=None, initial_data=None):
        super().__init__(parent)
        self.transient(parent)
        if title: self.title(title)
        self.parent = parent
        self.result = None
        self.initial_data = initial_data
        body = Frame(self)
        self.initial_focus = self._create_widgets(body)
        body.pack(padx=15, pady=15, side=tk.RIGHT)
        self.protocol("WM_DELETE_WINDOW", self._cancel)
        self.grab_set()
        if not self.initial_focus: self.initial_focus = self
        self.initial_focus.focus_set()
        self.wait_window(self)

    def _create_widgets(self, master):
        Label(master, text="請輸入註解:").pack(pady=(0, 5))
        self.entry = Entry(master, width=40)
        self.entry.pack(pady=(0, 15))
        if self.initial_data:
            self.entry.insert(0, self.initial_data.get("text", ""))
        button_frame = Frame(master)
        button_frame.pack()
        Button(button_frame, text="Stent", width=10, command=lambda: self._on_submit("Stent")).pack(side=tk.LEFT, padx=5)
        Button(button_frame, text="Balloon", width=10, command=lambda: self._on_submit("Balloon")).pack(side=tk.LEFT, padx=5)
        Button(button_frame, text="Cancel", width=10, command=self._cancel).pack(side=tk.LEFT, padx=5)
        return self.entry

    def _on_submit(self, annotation_type):
        text = self.entry.get().strip()
        if text:
            self.result = {"text": text, "type": annotation_type}
            self.destroy()
        else:
            self.entry.focus_set()

    def _cancel(self, event=None):
        self.result = None
        self.destroy()

In [46]:
class ImageAnnotator:
    def __init__(self, root, image_paths):
        self.root = root
        self.image_paths = image_paths
        # ... (其他屬性初始化) ...
        self.current_image_index = 0
        self.annotations = []
        self.displayed_annotations = []
        self.pil_image_original = None
        self.tk_image_displayed = None
        self.canvas_image_item = None
        self.display_width = 1
        self.display_height = 1
        self.current_image_scale_ratio = 1.0
        self.canvas_image_x_offset = 0
        self.canvas_image_y_offset = 0

        self.root.title("圖片標註工具")
        self._setup_ui()
        self._setup_listbox_menu()

        # 綁定事件
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.canvas.bind("<Configure>", self.on_canvas_resize)
        self.root.bind("<Left>", lambda event: self.prev_image())
        self.root.bind("<Right>", lambda event: self.next_image())
        self.annotation_listbox.bind("<Button-3>", self.show_listbox_menu)
        
        if self.image_paths:
            self.load_image_for_display()

    def _setup_ui(self):
        """設定使用者介面，包含新的'完成'按鈕。"""
        self.main_frame = Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        self.controls_frame = Frame(self.main_frame)
        self.controls_frame.pack(side=tk.TOP, fill=tk.X, pady=5)
        
        # 新增 '上一張,下一張,完成' 按鈕
        self.finish_button = Button(self.controls_frame, text="完成並匯出", command=self.finish_and_export, bg="#28a745", fg="white")
        self.finish_button.pack(side=tk.RIGHT, padx=10, pady=5)

        self.prev_button = Button(self.controls_frame, text="上一張 (←)", command=self.prev_image)
        self.prev_button.pack(side=tk.LEFT, padx=5)
        self.image_name_label = Label(self.controls_frame, text="", width=20, anchor="center")
        self.image_name_label.pack(side=tk.LEFT, expand=True, fill=tk.X)
        self.next_button = Button(self.controls_frame, text="下一張 (→)", command=self.next_image)
        self.next_button.pack(side=tk.RIGHT, padx=5)
        
        # ... 註解欄位 ...
        content_frame = Frame(self.main_frame)
        content_frame.pack(fill=tk.BOTH, expand=True)
        self.canvas = Canvas(content_frame, cursor="cross", bg="lightgrey")
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.annotation_panel = Frame(content_frame, width=250)
        self.annotation_panel.pack(side=tk.RIGHT, fill=tk.Y, padx=(5,0))
        Label(self.annotation_panel, text="已標註註解:").pack(pady=(5, 0), anchor='w')
        scrollbar = Scrollbar(self.annotation_panel)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.annotation_listbox = Listbox(self.annotation_panel, yscrollcommand=scrollbar.set)
        self.annotation_listbox.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=self.annotation_listbox.yview)

    def on_canvas_click(self, event):
        # ... (檢查點擊是否在圖片範圍內) ...
        if not self.tk_image_displayed or not self.pil_image_original: return
        img_w = self.tk_image_displayed.width()
        img_h = self.tk_image_displayed.height()
        if not (self.canvas_image_x_offset <= event.x < self.canvas_image_x_offset + img_w and
                self.canvas_image_y_offset <= event.y < self.canvas_image_y_offset + img_h):
            return

        dialog = CustomDialog(self.root, title="輸入註解")
        result = dialog.result
        if result:
            marker_color = "yellow" if result["type"] == "Balloon" else "red"
            image_x = event.x - self.canvas_image_x_offset
            image_y = event.y - self.canvas_image_y_offset
            self.annotations.append({
                "image_path": self.image_paths[self.current_image_index],
                "original_coords": (image_x / self.current_image_scale_ratio, image_y / self.current_image_scale_ratio),
                "text": result["text"], "type": result["type"], "color": marker_color,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 新增：記錄當下時間
            })
            self.redraw_annotations_for_current_image()
            self.update_annotation_list()

    def modify_selected_annotation(self):
        selected_indices = self.annotation_listbox.curselection()
        if not selected_indices: return
        anno_to_edit = self.displayed_annotations[selected_indices[0]]
        dialog = CustomDialog(self.root, title="修改註解", initial_data=anno_to_edit)
        result = dialog.result
        if result:
            anno_to_edit["text"] = result["text"]
            anno_to_edit["type"] = result["type"]
            anno_to_edit["color"] = "yellow" if result["type"] == "Balloon" else "red"
            anno_to_edit["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 新增：更新修改時間
            self.redraw_annotations_for_current_image()
            self.update_annotation_list()

    def finish_and_export(self):
        """完成標註，匯出圖片與Excel檔案，然後關閉程式。"""
        if not self.annotations:
            print("沒有任何標註，程式即將關閉。")
            self.root.destroy()
            return
        output_image = r"D:\report_ai\A00001"
        os.makedirs(output_image, exist_ok=True)

        print("開始匯出資料...")
        # 1. 將標註按圖片路徑分組
        grouped_annotations = {}
        for anno in self.annotations:
            path = anno['image_path']
            if path not in grouped_annotations:
                grouped_annotations[path] = []
            grouped_annotations[path].append(anno)
            
        # 2. 為每一張有標註的圖片進行匯出
        for image_path, annos_list in grouped_annotations.items():
            print(f"  正在處理圖片: {os.path.basename(image_path)}")
            
            # --- 匯出標註後的圖片 ---
            try:
                original_image = Image.open(image_path).convert("RGB")
                draw = ImageDraw.Draw(original_image)
                marker_radius = max(8, int(original_image.width / 200)) # 標記點大小隨圖片尺寸調整

                for anno in annos_list:
                    x, y = anno['original_coords']
                    color = anno['color']
                    draw.ellipse((x - marker_radius, y - marker_radius, x + marker_radius, y + marker_radius), 
                                 fill=color, outline="black", width=2)
                
                base, ext = os.path.splitext(image_path)
                output_image_path = os.path.join(output_image, f"{base}_annotated.png")
                original_image.save(output_image_path)
                print(f"    -> 標註圖片已儲存至: {output_image_path}")

            except Exception as e:
                print(f"    -> 儲存圖片時發生錯誤: {e}")

            # --- 匯出Excel檔案 ---
            export_data = []
            for i, anno in enumerate(annos_list, 1):
                export_data.append({
                    "編號": i,
                    "類型": anno['type'],
                    "註解": anno['text'],
                    "X座標": f"{anno['original_coords'][0]:.2f}",
                    "Y座標": f"{anno['original_coords'][1]:.2f}",
                    "標註時間": anno['timestamp']
                })
            
            df = pd.DataFrame(export_data)
            output_folder = r"D:\report_ai\A00001"
            os.makedirs(output_folder, exist_ok=True)
            output_excel_path = os.path.join(output_folder, f"{base}_annotations.xlsx")
            try:
                df.to_excel(output_excel_path, index=False, engine='openpyxl')
                print(f"    -> 標註資料已儲存至: {output_excel_path}")
            except Exception as e:
                print(f"    -> 儲存Excel時發生錯誤: {e}")

        print("所有資料匯出完成，程式即將關閉。")
        self.root.destroy()
    
    # --- 右鍵修改及刪除註解 ---
    def _setup_listbox_menu(self):
        self.listbox_menu = Menu(self.root, tearoff=0)
        self.listbox_menu.add_command(label="修改註解", command=self.modify_selected_annotation)
        self.listbox_menu.add_command(label="刪除註解", command=self.delete_selected_annotation)

    def show_listbox_menu(self, event):
        clicked_index = self.annotation_listbox.nearest(event.y)
        if clicked_index != -1:
            self.annotation_listbox.selection_clear(0, tk.END)
            self.annotation_listbox.selection_set(clicked_index)
            self.listbox_menu.tk_popup(event.x_root, event.y_root)

    def delete_selected_annotation(self):
        selected_indices = self.annotation_listbox.curselection()
        if not selected_indices: return
        anno_to_delete = self.displayed_annotations[selected_indices[0]]
        self.annotations.remove(anno_to_delete)
        self.redraw_annotations_for_current_image()
        self.update_annotation_list()

    def update_annotation_list(self):
        self.annotation_listbox.delete(0, tk.END)
        self.displayed_annotations = []
        current_image_basename = os.path.basename(self.image_paths[self.current_image_index])
        counter = 1
        for annotation in self.annotations:
            if os.path.basename(annotation["image_path"]) == current_image_basename:
                self.displayed_annotations.append(annotation)
                display_text = f"{counter}. [{annotation['type']}] {annotation['text']}"
                self.annotation_listbox.insert(tk.END, display_text)
                counter += 1

    def redraw_annotations_for_current_image(self):
        self.canvas.delete("annotation_marker")
        if not self.pil_image_original: return
        current_image_basename = os.path.basename(self.image_paths[self.current_image_index])
        for annotation in self.annotations:
            if os.path.basename(annotation["image_path"]) == current_image_basename:
                original_x, original_y = annotation["original_coords"]
                scaled_x = original_x * self.current_image_scale_ratio
                scaled_y = original_y * self.current_image_scale_ratio
                canvas_x = int(scaled_x + self.canvas_image_x_offset)
                canvas_y = int(scaled_y + self.canvas_image_y_offset)
                self.canvas.create_oval(canvas_x - 4, canvas_y - 4, canvas_x + 4, canvas_y + 4,
                                        fill=annotation["color"], outline=annotation["color"], tags="annotation_marker")

    def prev_image(self):
        if not self.image_paths: return
        self.current_image_index = (self.current_image_index - 1) % len(self.image_paths)
        self.load_image_for_display()

    def next_image(self):
        if not self.image_paths: return
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)
        self.load_image_for_display()
    
    # ... 當畫布改變大小,更新顯示高寬敝重新仔入圖片以適應大小 ...
    def on_canvas_resize(self, event):
        self.display_width = event.width
        self.display_height = event.height
        if self.pil_image_original: # 只有在已載入圖片時才重新調整
             self.display_image_on_canvas()
             self.redraw_annotations_for_current_image()

    def load_image_for_display(self):
        # 清除畫布上舊的標記
        image_path = self.image_paths[self.current_image_index]
        self.image_name_label.config(text=os.path.basename(image_path))
        try:
            self.pil_image_original = Image.open(image_path)
        except Exception as e:
            print(f"載入圖片失敗: {e}")
            self.canvas.delete("all")
            self.pil_image_original = None
            return
        self.display_image_on_canvas()
        self.redraw_annotations_for_current_image()
        self.update_annotation_list()

    def display_image_on_canvas(self):
        if not self.pil_image_original or self.display_width <= 1: return
        img_w, img_h = self.pil_image_original.size
        ratio = min(self.display_width / img_w, self.display_height / img_h)
        self.current_image_scale_ratio = ratio
        new_w, new_h = int(img_w * ratio), int(img_h * ratio)
        self.canvas_image_x_offset = (self.display_width - new_w) // 2
        self.canvas_image_y_offset = (self.display_height - new_h) // 2
        try: resample_filter = Image.Resampling.LANCZOS
        except AttributeError: resample_filter = Image.LANCZOS
        resized_pil_image = self.pil_image_original.resize((new_w, new_h), resample_filter)
        self.tk_image_displayed = ImageTk.PhotoImage(resized_pil_image)
        if self.canvas_image_item: self.canvas.delete(self.canvas_image_item)
        self.canvas_image_item = self.canvas.create_image(
            self.canvas_image_x_offset, self.canvas_image_y_offset,
            anchor=tk.NW, image=self.tk_image_displayed
        )

In [47]:
if __name__ == "__main__":
    image_files = ["L_side.jpg", "L_side_2.jpg", "R_side.png"]
    missing_files = [f for f in image_files if not os.path.exists(f)]
    if missing_files: # 檢查圖片是否存在
        print(f"警告: 以下圖片檔案不存在: {', '.join(missing_files)}")

    main_root = tk.Tk()
    main_root.geometry("1100x750") # 設定初始視窗大小
    app = ImageAnnotator(main_root, image_files)
    main_root.mainloop()

沒有任何標註，程式即將關閉。
